In [2]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import transforms, datasets, models, utils
from torch.utils.tensorboard import SummaryWriter
import time
import numpy as np
from torchsummary import summary
from torch.utils.data import DataLoader

In [3]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [4]:
def get_model():
    model = models.resnet50(pretrained=True)
    
    for param in model.parameters():
        param.requires_grad = False
    
    model.avgpool = AdaptiveConcatPool2d()
    model.fc = nn.Sequential(
        nn.Flatten(),
        nn.BatchNorm1d(4096),
        nn.Dropout(0.5),
        nn.Linear(4096, 512),
        nn.ReLU(),
        nn.BatchNorm1d(512),
        nn.Dropout(p=0.5),
        nn.Linear(512, 2),
        nn.LogSoftmax(dim=1)
    )
    return model

In [5]:
def train(model, device, train_loader, criterion, optimizer, epoch, writer):
    model.train()
    
    total_loss = 0
    
    for batch_id, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        preds = model(data)
        loss = criterion(preds, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    writer.add_scalar('Train Loss', total_loss/len(train_loader), epoch)
    writer.flush()
    
    return total_loss/len(train_loader)

In [6]:
def test(model, device, test_loader, criterion, epoch, writer):
    model.eval()
    
    total_loss, correct = 0, 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            total_loss += criterion(output, target).item()
            pred = output.data.max(1)[1]
            correct += pred.eq(target.data).cpu().sum()
            
            misclassified_images(pred, writer, target, data, output, epoch)
            
    total_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    writer.add_scalar('Test Loss', total_loss, epoch)
    writer.add_scalar('Accuracy', accuracy, epoch)
    writer.flush()
    
    return total_loss, accuracy

In [7]:
inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.255]
    )

In [8]:
def misclassified_images(pred, writer, target, data, output, epoch, count=10):
    misclassified = (pred != target.data)
    for index, image_tensor in enumerate(data[misclassified][:count]):
        img_name = '{}->Predict-{}x{}-Actual'.format(
                epoch,
                LABEL[pred[misclassified].tolist()[index]],
                LABEL[target.data[misclassified].tolist()[index]], 
            )
        
        writer.add_image(img_name, inv_normalize(image_tensor), epoch)

In [9]:
image_transforms = {
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=300, scale=(0.8, 1.1)),
        transforms.RandomRotation(degrees=10),
        transforms.ColorJitter(0.4, 0.4, 0.4),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=256),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    'val':
    transforms.Compose([
        transforms.Resize(size=300),
        transforms.CenterCrop(size=256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test':
    transforms.Compose([
        transforms.Resize(size=300),
        transforms.CenterCrop(size=256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [10]:
datadir = '../input/chest-xray-pneumonia/chest_xray/chest_xray/'
traindir = datadir + 'train/'
validdir = datadir + 'test/'
testdir = datadir + 'val/'

In [11]:
model_path = "model.pth"

In [12]:
batch_size = 128

In [13]:
data = {
    'train':
    datasets.ImageFolder(root=traindir, transform=image_transforms['train']),
    'val':
    datasets.ImageFolder(root=validdir, transform=image_transforms['val']),
    'test':
    datasets.ImageFolder(root=testdir, transform=image_transforms['test'])
}

In [14]:
dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'val': DataLoader(data['val'], batch_size=batch_size, shuffle=True),
    'test': DataLoader(data['test'], batch_size=batch_size, shuffle=True)
}

In [15]:
LABEL = dict((v,k) for k,v in data['train'].class_to_idx.items())

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
model = get_model().to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 74.3MB/s]


In [38]:
summary(model, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]           4,096
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          16,384
      BatchNorm2d-12          [-1, 256, 64, 64]             512
           Conv2d-13          [-1, 256, 64, 64]          16,384
      BatchNorm2d-14          [-1, 256,

In [18]:
criterion = nn.NLLLoss()

In [19]:
optimizer = optim.Adam(model.parameters())

In [20]:
PATH_to_log_dir = 'logdir/'

In [21]:
def tb_writer():
    timestr = time.strftime("%Y%m%d_%H%M%S")
    writer = SummaryWriter(PATH_to_log_dir + timestr)
    return writer

In [22]:
writer = tb_writer()
dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()
grid = utils.make_grid([inv_normalize(image) for image in images[:32]])
writer.add_image('X-Ray grid', grid, 0)
writer.flush()

In [23]:
def train_epochs(model, device, dataloaders, criterion, optimizer,epochs, writer):
    print('{0:>20} | {1:>20} | {2:>20} | {3:>20}  |'.format('Epoch','Training Loss','Test Loss', 'Accuracy'))
    best_score = np.Inf
    for epoch in epochs:
        train_loss = train(model, device, dataloaders['train'], criterion, optimizer, epoch, writer)
        test_loss, accuracy = test(model, device, dataloaders['val'], criterion, epoch, writer)
        if test_loss < best_score:
            best_score = test_loss
            torch.save(model.state_dict(), model_path)
        print('{0:>20} | {1:>20} | {2:>20} | {3:>20.2f}% |'.format(epoch,train_loss,test_loss, accuracy))
        writer.flush()

In [24]:
train_epochs(model, device, dataloaders, criterion, optimizer, range(0,10), writer)

               Epoch |        Training Loss |            Test Loss |             Accuracy  |
                   0 |  0.36176079947774004 |  0.49701107740402223 |                79.17% |
                   1 |   0.2337999732756033 |  0.38707311153411866 |                84.78% |
                   2 |  0.20143801955188192 |    0.370448511838913 |                85.26% |
                   3 |   0.1900221660006337 |   0.3626258075237274 |                86.86% |
                   4 |   0.1769030857376936 |    0.409450900554657 |                85.90% |
                   5 |   0.1636709040257989 |  0.36908947825431826 |                86.86% |
                   6 |  0.16074819299505977 |   0.4182650983333588 |                84.46% |
                   7 |  0.16151601621290532 |  0.40160723924636843 |                85.58% |
                   8 |  0.15197279885774706 |   0.4819407343864441 |                84.13% |
                   9 |  0.14418317559288768 |   0.4331103503704071 |  

In [25]:
writer.close()

In [26]:
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [27]:
def unfreeze(model):
    for param in model.parameters():
        param.requires_grad = True

In [28]:
unfreeze(model)

In [29]:
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=0.01)

In [30]:
writer = tb_writer()

In [31]:
train_epochs(model, device, dataloaders, criterion, optimizer, range(9,15), writer)

               Epoch |        Training Loss |            Test Loss |             Accuracy  |
                   9 |  0.15972968554351388 |  0.41342413425445557 |                85.10% |
                  10 |   0.1224460500042613 |   0.3801746487617493 |                86.54% |
                  11 |   0.1217333177422605 |  0.37790409922599794 |                87.18% |
                  12 |  0.11098722713749583 |   0.3712982594966888 |                87.98% |
                  13 |  0.09877484373566581 |  0.41088773012161256 |                86.70% |
                  14 |  0.09256085244620718 |   0.3181425631046295 |                89.42% |


In [32]:
writer.close()